## A re-Tweet Graph

Adapting the classic HITS approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their retweets of other Twitter users (so user = node, retweet of another user = edge). Over this Twitter-user graph, we can apply the HITS approach to order the users by their hub-ness and their authority-ness.

Here is a toy example. Suppose you are given the following four retweets:

* **userID**: diane, **text**: "RT ", **sourceID**: bob
* **userID**: charlie, **text**: "RT Welcome", **sourceID**: alice
* **userID**: bob, **text**: "RT Hi ", **sourceID**: diane
* **userID**: alice, **text**: "RT Howdy!", **sourceID**: parisa

There are four short tweets retweeted by four users. The retweet between users form a directed graph with five nodes and four edges. E.g., the "diane" node has a directed edge to the "bob" node.

Building a graph by parsing the tweets in the file we provide called *HITS.json*.

**Notes:**
* The edges are weighted and directed. If Bob retweets Alice's tweets 10 times, there is an edge from Bob to Alice with weight 10, but there is not an edge from Alice to Bob.
* If a user retweets herself, ignore it.
* Correctly parsing screen_name in a tweet is error-prone. Use the id of the user (this is the user who is re-tweeting) and the id of the user in the retweeted_status field (this is the user who is being re-tweeted; that is, this user created the original tweet).

In [1]:
from collections import defaultdict
import networkx as nx
import jsonlines
from collections import Counter
import numpy as np
from numpy import linalg as LA
import operator

This method will read the input file containing bunch of tweet jsons. Using these tweets, it will create a tweet graph. 

In [2]:
def create_tweet_graph_from_file(filename):
    edges_list = [] ## This list contains all the directed edges in the graph

    ## Reading the input json file
    with jsonlines.open(filename, 'r') as f:
        for jsn in f:
            rt_user_id = jsn["user"]["id"]
            source_user_id = jsn["retweeted_status"]["user"]["id"]
            if rt_user_id != source_user_id:
                edges_list.append((rt_user_id, source_user_id))
    
    ## Creating a weighted edge list from the edges_list
    weighted_edge_list = Counter(edges_list)
    tweet_graph = nx.DiGraph() ## Creating an empty directed graph
    
    # Adding edges to the directed graph from the weighted edges list
    for edge in weighted_edge_list.items():
        source = edge[0][0]
        destination = edge[0][1]
        weight = edge[1]
        tweet_graph.add_edge(source, destination, weight=weight)
    return tweet_graph

In [3]:
tweet_graph = create_tweet_graph_from_file('HITS.json')
tweet_graph.size()

6177

## HITS Implementation

This program will return the top 10 users with highest hub and authority scores. The **output** is like:

Hub Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

Authority Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

#### Approach : Using the power iteration method

In [6]:
# Given input graph, this method is the implementation of hits algorithms.
# It returns the hubs and authorities score

def hits(graph, iter_count = 20):
    nodes = graph.nodes()
    nodes_count = len(nodes)
    matrix = nx.to_numpy_matrix(graph, nodelist=nodes)
    
    hubs_score = np.ones(nodes_count)
    auth_score = np.ones(nodes_count)
    H = matrix * matrix.T
    A = matrix.T * matrix
   
    for i in range(iter_count):
       
        hubs_score = hubs_score * H 
        auth_score = auth_score * A 
        hubs_score = hubs_score / LA.norm(hubs_score)
        auth_score = auth_score / LA.norm(auth_score)
        
    hubs_score = np.array(hubs_score).reshape(-1,)
    auth_score = np.array(auth_score).reshape(-1,)
    
    hubs = dict(zip(nodes, hubs_score))
    authorities = dict(zip(nodes, auth_score))
    return hubs, authorities

In [7]:
# Given a graph, this method returns top k hubs
def get_top_k_hubs(graph, k = 10):
    hubs = hits(graph)[0]
    return sorted(hubs.items(), key = operator.itemgetter(1), reverse = True)[:k]

#Given a graph, this method returns top k authorities
def get_top_k_authorities(graph, k = 10):
    auth = hits(graph)[1]
    return sorted(auth.items(), key = operator.itemgetter(1), reverse = True)[:k]

In [8]:
top_10_tweet_hubs = get_top_k_hubs(tweet_graph)
print("Top 10 hubs ")
top_10_tweet_hubs

Top 10 hubs 


[(3068706044, 0.6228962788346416),
 (3093940760, 0.29608337726157852),
 (2194518394, 0.25979684894330446),
 (2862783698, 0.20250708715416696),
 (3092183276, 0.17046401522271865),
 (3029724797, 0.16693938874412703),
 (2990704188, 0.14781712484957887),
 (3001500121, 0.14506944928145832),
 (3086921438, 0.12911896850758384),
 (3042686360, 0.12523755718547352)]

In [9]:
top_10_tweet_auth = get_top_k_authorities(tweet_graph)
print("Top 10 authorities")
top_10_tweet_auth

Top 10 authorities


[(3042570996, 0.54450841838976316),
 (3065514742, 0.49299557729892024),
 (1638625987, 0.44375892390040017),
 (3077733683, 0.28651236641693456),
 (3039321886, 0.22426278276458395),
 (3077695572, 0.12184230146264768),
 (3019659587, 0.11321175872457195),
 (1358345766, 0.098942091627440804),
 (3061155846, 0.09396927090171632),
 (3092580049, 0.093661391183235035)]